In [ ]:
!pip install "anomalib[vlm_clip]"
!pip install open-clip-torch


In [ ]:
from anomalib.models.image import Stfpm
from anomalib.engine import Engine
from anomalib.data import MVTecAD

In [ ]:
import os, glob
from PIL import Image

base = "/content/drive/MyDrive/Colab Notebooks/data/battery_small"
folders = [
    f"{base}/train/good",
    f"{base}/test/good",
    f"{base}/test/overheat",
    f"{base}/test/thermal_imbalance",
    f"{base}/test/spatial_tape",
    f"{base}/ground_truth/overheat",
    f"{base}/ground_truth/thermal_imbalance",
    f"{base}/ground_truth/spatial_tape"
]

for folder in folders:
    if not os.path.isdir(folder):
        continue
    for p in glob.glob(os.path.join(folder, "*.jpg")) + glob.glob(os.path.join(folder, "*.JPG")):
        out = os.path.splitext(p)[0] + ".png"
        if not os.path.exists(out):
            Image.open(p).convert("RGB").save(out)
print("Done converting to PNG.")

In [ ]:
 Initialize model and data
datamodule = MVTecAD(
    root="/content/drive/MyDrive/Colab Notebooks/data",
    category="battery_small",       # folder under your_dataset/

)
datamodule.setup()
print("Train samples:", len(datamodule.train_data.samples))
print("Test samples:", len(datamodule.test_data.samples))

In [ ]:
datamodule = MVTecAD(
    root="/content/drive/MyDrive/Colab Notebooks/data",
    category="battery_small",

)
datamodule.setup()
print("Train samples:", len(datamodule.train_data.samples))
print("Test samples:", len(datamodule.test_data.samples))

In [ ]:
model = Stfpm(
    backbone="resnet18",
    layers=["layer1", "layer2", "layer3"]
)

In [ ]:
engine = Engine(model=model, datamodule=datamodule)
engine.fit()

In [ ]:
predictions = engine.predict(model, datamodule=datamodule)

In [ ]:
test_metrics = engine.test(model=model, datamodule=datamodule)
print(test_metrics)
